In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
from google.colab import files

In [6]:
files.upload() 

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kenseismart","key":"f0770d64c231bec5e129fd571312b5e5"}'}

In [7]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 Nov 16 10:54 kaggle.json


In [8]:
!pip install -q kaggle

In [9]:
!mkdir -p ~/.kaggle

In [10]:
!cp kaggle.json ~/.kaggle/

In [11]:
!chmod 600 /root/.kaggle/kaggle.json

In [12]:
!pwd

/content


In [13]:
!kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-11-10 18:11:20          15659  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           9487  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           4273  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3475  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [23]:
!kaggle competitions download -c avito-demand-prediction  -f train.csv

 98% 313M/318M [00:02<00:00, 134MB/s]
100% 318M/318M [00:02<00:00, 115MB/s]


In [24]:
!  python -m pip install dask[dataframe] --upgrade
import dask.dataframe as dd
import pandas as pd

     |████████████████████████████████| 132 kB 5.1 MB/s 


In [25]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [26]:
train_df = pd.read_csv('train.csv', error_bad_lines=False, engine="python")

In [27]:
train_df.head(1)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789


## Data cleaning

In [28]:
len(train_df)

1503424

## I will drop item_id, because they are all unique.




In [29]:
train_df = train_df.drop(['item_id'], axis=1)

## LobalCodeEncoding for user_id

In [30]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [31]:
all_users = train_df.user_id

In [32]:
all_users = labelencoder.fit_transform(all_users)

In [33]:
user_idx = train_df.user_id

In [34]:
user_idx[:6]

0    e00f8ff2eaf9
1    39aeb48f0017
2    91e2f88dd6e3
3    bf5cccea572d
4    ef50846afc0b
5    bbfad0b1ad0a
Name: user_id, dtype: object

In [35]:
train_df.user_id = all_users

In [36]:
train_df.user_id.nunique()

771769

In [37]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,173962,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,440069,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


LabelCodeEncoding for city

In [38]:
city_encoder = LabelEncoder()

In [39]:
train_df.city.isna().sum()

0

In [40]:
train_df.city = city_encoder.fit_transform(train_df.city)

In [41]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,Свердловская область,460,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,173962,Самарская область,1300,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,440069,Ростовская область,1276,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


## LabelEncoding For region

In [42]:
region_encoder = LabelEncoder()

In [43]:
train_df.region = region_encoder.fit_transform(train_df.region)

In [44]:
train_df.head()

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,19,460,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,173962,17,1300,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,440069,16,1276,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,576929,21,940,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,721853,4,317,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


## LabelEncoding For parent_category_name

In [45]:
train_df.parent_category_name.isna().sum()

0

In [46]:
parent_category_name_encoder = LabelEncoder()

In [47]:
train_df.parent_category_name = parent_category_name_encoder.fit_transform(train_df.parent_category_name)

In [48]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,19,460,4,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,173962,17,1300,2,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,440069,16,1276,0,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


LabelEncoding for category_name

In [49]:
train_df.category_name.isna().sum()

0

In [50]:
category_name_encoder = LabelEncoder()

In [51]:
train_df.category_name = category_name_encoder.fit_transform(train_df.category_name)

In [52]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,19,460,4,42,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,173962,17,1300,2,22,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,440069,16,1276,0,2,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


LabelEncoder for param

In [53]:
train_df.param_1.isna().sum()

61576

In [54]:
train_df.param_1.isna().sum() / (len(train_df))

0.0409571750883317

In [55]:
train_df['param_1'].value_counts().idxmax()

'Женская одежда'

In [56]:
train_df.param_1 = train_df.param_1.fillna('Женская одежда')

In [57]:
param_1_encoder = LabelEncoder()

In [58]:
train_df.param_1 = param_1_encoder.fit_transform(train_df.param_1)

In [59]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,19,460,4,42,248,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,173962,17,1300,2,22,121,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,440069,16,1276,0,2,83,NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


In [60]:
train_df.param_2.isna().sum()

654542

In [61]:
train_df.param_3.isna().sum()

862565

In [62]:
train_df.param_2 = train_df.param_2.fillna('Zero')
train_df.param_3 = train_df.param_3.fillna('Zero')

In [63]:
param_2_encoder = LabelEncoder()
param_3_encoder = LabelEncoder()

In [64]:
train_df.param_2 = param_2_encoder.fit_transform(train_df.param_2)
train_df.param_3 = param_3_encoder.fit_transform(train_df.param_3)

In [65]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,19,460,4,42,248,107,1149,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,173962,17,1300,2,22,121,107,1149,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,440069,16,1276,0,2,83,107,1149,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177


# OTHERS Columns

In [66]:
train_df.title.nunique()

788377

In [67]:
train_df.title.isna().sum()

0

In [68]:
titel_encoder = LabelEncoder()

In [69]:
train_df.title = titel_encoder.fit_transform(train_df.title)

In [70]:
train_df = train_df.drop(['description'], axis=1)
train_df.head(1)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,675853,19,460,4,42,248,107,1149,298998,400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789


In [71]:
train_df = train_df.drop(['image'], axis=1)
train_df.head(1)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,price,item_seq_number,activation_date,user_type,image_top_1,deal_probability
0,675853,19,460,4,42,248,107,1149,298998,400.0,2,2017-03-28,Private,1008.0,0.12789


In [72]:
train_df.user_type.isna().sum()

0

In [73]:
type_encoder = LabelEncoder()
train_df.user_type = type_encoder.fit_transform(train_df.user_type)
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,price,item_seq_number,activation_date,user_type,image_top_1,deal_probability
0,675853,19,460,4,42,248,107,1149,298998,400.0,2,2017-03-28,1,1008.0,0.12789
1,173962,17,1300,2,22,121,107,1149,676606,3000.0,19,2017-03-26,1,692.0,0.00000
2,440069,16,1276,0,2,83,107,1149,79843,4000.0,9,2017-03-20,1,3032.0,0.43177


In [74]:
train_df.activation_year = [i.split('-')[0] for i in train_df.activation_date]
train_df.activation_month = [i.split('-')[1] for i in train_df.activation_date]
train_df.activation_data = [i.split('-')[2] for i in train_df.activation_date]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [75]:
# Del this column and find that other date column were disappeared, got that and keep move foraward.
train_df = train_df.drop(['activation_date'], axis=1)
train_df.head(1)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,price,item_seq_number,user_type,image_top_1,deal_probability
0,675853,19,460,4,42,248,107,1149,298998,400.0,2,1,1008.0,0.12789


In [76]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,price,item_seq_number,user_type,image_top_1,deal_probability
0,675853,19,460,4,42,248,107,1149,298998,400.0,2,1,1008.0,0.12789
1,173962,17,1300,2,22,121,107,1149,676606,3000.0,19,1,692.0,0.00000
2,440069,16,1276,0,2,83,107,1149,79843,4000.0,9,1,3032.0,0.43177


# train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [77]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_df, test_size=0.2, random_state=13)

# Создать Dataset для загрузки данных (используем только числовые данные)

In [78]:
import math
import torch
import numpy as np

import numpy as np

from PIL import Image
from torchvision import transforms, datasets

In [79]:
train_df.head(3)

,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,price,item_seq_number,user_type,image_top_1,deal_probability
0,675853,19,460,4,42,248,107,1149,298998,400.0,2,1,1008.0,0.12789
1,173962,17,1300,2,22,121,107,1149,676606,3000.0,19,1,692.0,0.00000
2,440069,16,1276,0,2,83,107,1149,79843,4000.0,9,1,3032.0,0.43177


In [80]:
torch.tensor(train_df.drop(['deal_probability'], axis=1).values.astype(np.float32))

tensor([[6.7585e+05, 1.9000e+01, 4.6000e+02,  ..., 2.0000e+00, 1.0000e+00,
         1.0080e+03],
        [1.7396e+05, 1.7000e+01, 1.3000e+03,  ..., 1.9000e+01, 1.0000e+00,
         6.9200e+02],
        [4.4007e+05, 1.6000e+01, 1.2760e+03,  ..., 9.0000e+00, 1.0000e+00,
         3.0320e+03],
        ...,
        [3.0980e+05, 1.0000e+00, 1.3700e+02,  ..., 5.0000e+01, 0.0000e+00,
         1.2020e+03],
        [1.3626e+05, 1.5000e+01, 1.5500e+02,  ..., 3.3000e+01, 1.0000e+00,
         8.7000e+01],
        [7.3351e+05, 9.0000e+00, 4.7000e+01,  ..., 1.8000e+01, 1.0000e+00,
         9.7900e+02]])

In [81]:
class MyDataset(torch.utils.data.Dataset):
  
    def __init__(self, X, y):
        self.X = torch.tensor(X.values.astype(np.float32))
        self.y = torch.tensor(y.values.astype(np.float32))

    def __len__(self):
        return len(self.X)
  
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [82]:
dataset = MyDataset(X=train.drop(['deal_probability'], axis=1), y=train['deal_probability'])

In [83]:
train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2, drop_last=True)

In [99]:
for batch, label in train_loader:
    print(batch.shape)
    print(label)
    break

torch.Size([64, 13])
tensor([0.2000, 0.0000, 0.0000, 0.6000, 0.8652, 0.0000, 0.0000, 0.3605, 0.0000,
        0.3272, 0.0991, 0.0000, 0.0000, 0.7040, 0.1721, 0.0000, 0.1059, 0.1236,
        0.0000, 0.2967, 0.2389, 0.4457, 0.0000, 0.7679, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.7679, 0.7040, 0.1540, 0.0000, 0.0996, 0.0000,
        0.0416, 0.0000, 0.0000, 0.8806, 0.0000, 0.0000, 0.8032, 0.0000, 0.0000,
        0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8032, 0.8652, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.1581, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])


In [85]:
test_dataset = MyDataset(X=test.drop(['deal_probability'], axis=1), y=test['deal_probability'])

In [86]:
test_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2, drop_last=True)

In [98]:
for batch, label in train_loader:
    print(batch.shape)
    print(label)
    break

torch.Size([64, 13])
tensor([0.0000, 0.1872, 0.8652, 0.0000, 0.0000, 0.1287, 0.0000, 0.7500, 0.1206,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.8652, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1286, 0.0000, 0.0000,
        0.0000, 0.0974, 0.0000, 0.0000, 0.0000, 0.8032, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0772, 0.0000, 0.0000, 0.8032, 0.8267, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.3397, 0.1026, 0.0000, 0.0000, 0.2120, 0.0000,
        0.8032, 0.0000, 0.0000, 0.2265, 0.1075, 0.0527, 0.0000, 0.0000, 0.0000,
        0.0000])


In [134]:
# Хотя как писал конфуций перед тем как засунуть в нейронку данные нормализуЙ! я поругав себя очень сильно продолжу просто так.
import torch.nn as nn

ReLU = nn.ReLU()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 50)
        self.dp = nn.Dropout(0.25)
        self.fc3 = nn.Linear(50, 1)


    def forward(self, x):
        x = self.fc1(x)
        # x = self.bn1(x)
        # x = ReLU(self.fc2(x))
        # x = self.dp(x)
        x = self.fc3(x)
        return x.view(-1)

In [135]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=13, out_features=50, bias=True)
  (bn1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (dp): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=50, out_features=1, bias=True)
)


In [118]:
x = batch
x.shape

torch.Size([64, 13])

Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать

In [119]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()

    def forward(self,x,y):
        criterion = nn.MSELoss()
        loss = torch.sqrt(criterion(x, y))
        return loss

In [125]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-10)
loss_fn = RMSELoss()

In [121]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()

        # pred = model(X)
        # loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [122]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [123]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [136]:
loss_fn = nn.BCEWithLogitsLoss()
epochs = 10000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, net, loss_fn, optimizer)
    test(test_loader, net, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:     nan  [    0/1202739]
loss:     nan  [ 6400/1202739]
loss: 13345.139648  [12800/1202739]
loss:     nan  [19200/1202739]
loss:     nan  [25600/1202739]
loss:     nan  [32000/1202739]
loss:     nan  [38400/1202739]
loss:     nan  [44800/1202739]
loss:     nan  [51200/1202739]
loss:     nan  [57600/1202739]
loss:     nan  [64000/1202739]
loss:     nan  [70400/1202739]
loss:     nan  [76800/1202739]
loss:     nan  [83200/1202739]
loss:     nan  [89600/1202739]
loss:     nan  [96000/1202739]
loss:     nan  [102400/1202739]
loss:     nan  [108800/1202739]
loss:     nan  [115200/1202739]
loss:     nan  [121600/1202739]
loss:     nan  [128000/1202739]
loss:     nan  [134400/1202739]
loss:     nan  [140800/1202739]
loss:     nan  [147200/1202739]
loss:     nan  [153600/1202739]
loss:     nan  [160000/1202739]
loss:     nan  [166400/1202739]
loss:     nan  [172800/1202739]
loss:     nan  [179200/1202739]
loss:     nan  [185600/1202739]
loss:     n

IndexError: ignored